# Match do emprego

In [ ]:
import pandas as pd
import numpy as np

: 

In [ ]:
# df_vagas = pd.read_csv('data/D_ETL_IMO_EXTRACAO_SINE_ABERTO_VAGAS_PR.csv', encoding = "ISO-8859-1",sep=';')
df_vagas = pd.read_excel('data/D_ETL_IMO_EXTRACAO_SINE_ABERTO_VAGAS_PR.xlsx')
pd.set_option('max_colwidth', None)
df_vagas.head()

: 

### Pré-processamento vagas 
- tipo_contratacao e escolaridade: substituindo valores textuais para numéricos
- qtd_experiencia: preenchendo campos vazios com zero e convertendo campo float para int**

In [ ]:
# Remove as colunas que não serão utilizadas
df_vagas=df_vagas.drop(['REQUERIDO_DEFICIENCIA', 'DEFICIENCIAS','CEP','OBSERVACOES_OCUPACAO','TITULO_OCUPACAO','QTD_VAGAS','CODIGO_MUNICIPIO_IBGE'], axis=1)

df_vagas.rename(columns={'CODIGO_OCUPACAO': 'cod_ocupacao', 'VALOR_SALARIO': 'salario', 'TIPO_CONTRATACAO': 'tipo_contratacao',
'QTD_EXPERIENCIA': 'qtd_experiencia', 'ESCOLARIDADE': 'escolaridade'}, inplace=True)

: 

In [ ]:

# Tipo contratacao
df_vagas.loc[df_vagas['tipo_contratacao'] == '0 - Estágio', 'tipo_contratacao']     = 0
df_vagas.loc[df_vagas['tipo_contratacao'] == '1 - Aprendiz', 'tipo_contratacao']    = 1 
df_vagas.loc[df_vagas['tipo_contratacao'] == '2 - Temporária', 'tipo_contratacao']  = 2
df_vagas.loc[df_vagas['tipo_contratacao'] == '3 - Permanente', 'tipo_contratacao']  = 3

# Escolaridade
df_vagas.loc[df_vagas['escolaridade'] == '0 - Nenhum', 'escolaridade']  = 0
df_vagas.loc[df_vagas['escolaridade'] == '10 - Fundamental Incompleto', 'escolaridade']  = 1
df_vagas.loc[df_vagas['escolaridade'] == '5 - Fundamental Completo', 'escolaridade']  = 2
df_vagas.loc[df_vagas['escolaridade'] == '6 - Médio Incompleto', 'escolaridade']  = 3
df_vagas.loc[df_vagas['escolaridade'] == '7 - Médio Completo', 'escolaridade']  = 4
df_vagas.loc[df_vagas['escolaridade'] == '8 - Superior Incompleto', 'escolaridade']  = 5
df_vagas.loc[df_vagas['escolaridade'] == 'Superior Completo', 'escolaridade']  = 6
df_vagas.loc[df_vagas['escolaridade'] == '13 - Especialização', 'escolaridade']  = 7

# Qtd. Experiência
df_vagas.fillna(0, inplace=True)
df_vagas.qtd_experiencia =df_vagas.qtd_experiencia.astype(int)

df_vagas['cod_ocupacao'] = df_vagas.cod_ocupacao.astype(np.int64)
df_vagas['salario'] = df_vagas.salario.astype(np.int64)

df_vagas.to_excel('data/vagas-pr.xlsx')
df_vagas

: 

### Pré-processamento candidatos
- deficiencia: 0 (não), 1 (sim)
- escolaridade: domínio de -1 (Não Identificado) a 8 (Doutorado)

In [ ]:
df_candidatos = pd.read_excel('data/candidatos-pr.xlsx')
df_candidatos.head() #2m 53.6s

: 

In [ ]:
import re 

def getOcupacoes(txtOcupacoes):
    pattern_cod_ocupacao = '[\d]{6}' 

    ocupacoes = txtOcupacoes.split('|')
    cod_ocupacao = []

    for partOcupacao in ocupacoes:
        print('Ocupação pretendida: ' + partOcupacao)
        cod_ocupacao += re.findall(pattern_cod_ocupacao, partOcupacao)

    return cod_ocupacao


: 

In [ ]:
# Deficiência
df_candidatos.loc[df_candidatos['deficiencia'] == '', 'deficiencia'] = 0
df_candidatos.loc[df_candidatos['deficiencia'] != '', 'deficiencia'] = 1

# Escolaridade
df_candidatos.loc[df_candidatos['escolaridade'] == 'Não Identificado', 'escolaridade']  = -1
df_candidatos.loc[df_candidatos['escolaridade'] == 'Nenhum', 'escolaridade']  = 0
df_candidatos.loc[df_candidatos['escolaridade'] == 'Analfabeto', 'escolaridade']  = 0
df_candidatos.loc[df_candidatos['escolaridade'] == 'Fundamental Incompleto', 'escolaridade']  = 1
df_candidatos.loc[df_candidatos['escolaridade'] == 'Fundamental Completo', 'escolaridade']  = 2
df_candidatos.loc[df_candidatos['escolaridade'] == 'Médio Incompleto', 'escolaridade']  = 3
df_candidatos.loc[df_candidatos['escolaridade'] == 'Médio Completo', 'escolaridade']  = 4
df_candidatos.loc[df_candidatos['escolaridade'] == 'Superior Incompleto', 'escolaridade']  = 4
df_candidatos.loc[df_candidatos['escolaridade'] == 'Superior Completo', 'escolaridade']  = 5
df_candidatos.loc[df_candidatos['escolaridade'] == 'Especialização', 'escolaridade']  = 6
df_candidatos.loc[df_candidatos['escolaridade'] == 'Mestrado', 'escolaridade']  = 7
df_candidatos.loc[df_candidatos['escolaridade'] == 'Doutorado', 'escolaridade']  = 8

# Estudante
df_candidatos.loc[df_candidatos['estudante'] == 'N', 'estudante'] = 0
df_candidatos.loc[df_candidatos['estudante'] == 'S', 'deficiencia'] = 1


# Habilitacao
from sklearn.preprocessing import LabelEncoder
labelencoder_X = LabelEncoder()

habilitacao_X = df_candidatos.loc[:, 'habilitacao'] # column habilitacao
df_candidatos.loc[:, 'habilitacao'] = labelencoder_X.fit_transform(habilitacao_X)

# Veículo
df_candidatos.loc[df_candidatos['veiculo'] == 'N', 'veiculo'] = 0
df_candidatos.loc[df_candidatos['veiculo'] == 'S', 'veiculo'] = 1

# Disponibilidade viajar
df_candidatos.loc[df_candidatos['disp_viajar'] == 'N', 'disp_viajar'] = 0
df_candidatos.loc[df_candidatos['disp_viajar'] == 'S', 'disp_viajar'] = 1

# Disponibilidade dormir
df_candidatos.loc[df_candidatos['disp_dormir'] == 'N', 'disp_dormir'] = 0
df_candidatos.loc[df_candidatos['disp_dormir'] == 'S', 'disp_dormir'] = 1

# Disponibilidade ausentar domicilio
df_candidatos.loc[df_candidatos['disp_ausentar_domicilio'] == 'N', 'disp_ausentar_domicilio'] = 0
df_candidatos.loc[df_candidatos['disp_ausentar_domicilio'] == 'S', 'disp_ausentar_domicilio'] = 1

# Ocupacao pretendida
# retirando linhas cuja ocupacao_pretendida não tem valor
# df_candidatos = df_candidatos[df_candidatos['ocupacao_pretendida'].notna()]
df_candidatos = df_candidatos[df_candidatos.ocupacao_pretendida != '']

df_candidatos.count()

for index, row in df_candidatos.iterrows():    
    df_candidatos.at[index, 'ocupacao_pretendida'] = getOcupacoes(row['ocupacao_pretendida'].strip())


# df_candidatos.to_excel('data/candidatos-pr.xlsx')
# df_candidatos

: 